# 【ディスク節約・最終版】AI学習ノートブック

**目的：RAM不足、接続エラー、ディスク容量不足の3大問題を完全に解決するため、処理済みのチャンクファイルを即座に削除する「追記＆削除」方式を採用します。これが無料版Colabでプロジェクトを完遂させるための最も堅牢なアプローチです。**

---

## STEP 1: 環境設定と再接続（最重要）

**最初に必ず実行してください。**
Google Driveとの接続を確立し、作業場所へ移動します。

---
**【エラーが起きたら？】**
もし、この後のステップでエラーが出た場合は、**慌てずにこのSTEP 1のセルをもう一度実行してください。** 接続がリフレッシュされ、安全に作業を再開できます。
---

In [1]:
!pip install mahjong
from google.colab import drive
# force_remount=True で、壊れた接続を強制的にリフレッシュします
drive.mount('/content/drive', force_remount=True)

# Google Drive上のプロジェクトルートディレクトリを定義します
GDRIVE_PROJECT_ROOT = "/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI"

# プロジェクトのルートディレクトリに移動します
%cd {GDRIVE_PROJECT_ROOT}

Mounted at /content/drive
/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI


## STEP 2: 【追記＆削除方式】チャンクを結合しては削除

**STEP 1の完了後、このセルを実行してください。**
チャンクファイルを一つずつバイナリファイルに追記し、成功したら即座に元のチャンクファイルを削除します。これにより、ディスク使用量を増やすことなく処理を進めます。

**（注意：完了までには非常に長い時間がかかりますが、放置しておけばいつか終わります）**

In [ ]:
import glob, pickle, numpy as np, os, tensorflow as tf

#【安全対策】実行前に必ずプロジェクトルートに移動します
%cd {GDRIVE_PROJECT_ROOT}

# 各種パスを設定します
GDRIVE_PROCESSED_DIR = os.path.join(GDRIVE_PROJECT_ROOT, 'processed_data')
CHUNK_DIR_GDRIVE = os.path.join(GDRIVE_PROCESSED_DIR, 'chunks')
MERGED_LOG_PATH = os.path.join(GDRIVE_PROCESSED_DIR, 'merged_chunks.log')
CONTEXTS_BIN_PATH = os.path.join(GDRIVE_PROCESSED_DIR, 'contexts.bin')
CHOICES_BIN_PATH = os.path.join(GDRIVE_PROCESSED_DIR, 'choices.bin')
LABELS_BIN_PATH = os.path.join(GDRIVE_PROCESSED_DIR, 'labels.bin')

print("--- Starting Append-and-Delete Merging Process ---")
all_chunk_files = sorted(glob.glob(os.path.join(CHUNK_DIR_GDRIVE, "*.pkl")))

if not all_chunk_files:
    print("No chunk files found. Assuming merge is complete. Proceed to the next step.")
else:
    # 既にマージ済みのファイルリストを読み込む
    merged_files = set()
    if os.path.exists(MERGED_LOG_PATH):
        with open(MERGED_LOG_PATH, 'r') as f:
            merged_files = set(line.strip() for line in f)

    files_to_merge = [f for f in all_chunk_files if f not in merged_files]

    if not files_to_merge:
        print("All chunks have already been merged! Proceed to the next step.")
    else:
        print(f"{len(merged_files)} chunks already processed. Resuming with {len(files_to_merge)} remaining chunks.")

        progbar = tf.keras.utils.Progbar(len(files_to_merge), unit_name="chunk")
        with open(MERGED_LOG_PATH, 'a') as log_f:
            for i, chunk_file in enumerate(files_to_merge):
                try:
                    # 1. チャンクを読み込む
                    with open(chunk_file, 'rb') as f:
                        contexts, choices, labels = pickle.load(f)

                    # 2. バイナリファイルに追記
                    with open(CONTEXTS_BIN_PATH, 'ab') as f_ctx, open(CHOICES_BIN_PATH, 'ab') as f_cho, open(LABELS_BIN_PATH, 'ab') as f_lbl:
                        contexts.tofile(f_ctx)
                        choices.tofile(f_cho)
                        labels.tofile(f_lbl)

                    # 3. 成功ログを記録
                    log_f.write(chunk_file + '\n')
                    log_f.flush()

                    # 4. 元のチャンクファイルを削除
                    os.remove(chunk_file)

                    progbar.update(i + 1)
                except Exception as e:
                    print(f"\nAn error occurred while processing {os.path.basename(chunk_file)}: {e}")
                    print("Please re-run STEP 1 and then this cell to resume.")
                    # エラー発生時は安全のため処理を中断
                    raise

        print("\n--- Append & Delete Merging Complete! Proceed to Final Formatting. ---")

/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI
--- Starting Append-and-Delete Merging Process ---


## STEP 3: 最終データ整形 ＆ データセット完成

**STEP 2が完了したら、このセルを実行してください。**
追記して作成した巨大なバイナリファイルを読み込み、最終的な学習データセットを完成させます。

In [ ]:
import pickle, numpy as np, os
from src.transformer.vectorizer import MAX_CONTEXT_LENGTH, MAX_CHOICES, VECTOR_DIM

#【安全対策】実行前に必ずプロジェクトルートに移動します
%cd {GDRIVE_PROJECT_ROOT}

# 各種パスを設定します
GDRIVE_PROCESSED_DIR = os.path.join(GDRIVE_PROJECT_ROOT, 'processed_data')
FINAL_DATASET_PATH_GDRIVE = os.path.join(GDRIVE_PROCESSED_DIR, 'training_dataset_transformer.pkl')
CONTEXTS_BIN_PATH = os.path.join(GDRIVE_PROCESSED_DIR, 'contexts.bin')
CHOICES_BIN_PATH = os.path.join(GDRIVE_PROCESSED_DIR, 'choices.bin')
LABELS_BIN_PATH = os.path.join(GDRIVE_PROCESSED_DIR, 'labels.bin')

print("--- Starting Final Data Formatting ---")
if not os.path.exists(LABELS_BIN_PATH):
    print("Error: Merged binary file not found. Please complete STEP 2 first.")
else:
    # バイナリファイルからnumpy配列として読み込む
    print("Loading binary data files...")
    contexts_flat = np.fromfile(CONTEXTS_BIN_PATH, dtype=np.float32)
    choices_flat = np.fromfile(CHOICES_BIN_PATH, dtype=np.float32)
    labels_final = np.fromfile(LABELS_BIN_PATH, dtype=np.int32)

    # 正しい形状にリシェイプ
    total_samples = len(labels_final)
    contexts_final = contexts_flat.reshape(total_samples, MAX_CONTEXT_LENGTH, VECTOR_DIM)
    choices_final = choices_flat.reshape(total_samples, MAX_CHOICES, VECTOR_DIM)

    print(f"Loaded and reshaped data with {total_samples} samples.")
    print("Generating final masks...")

    non_zero_counts = np.count_nonzero(np.sum(choices_final, axis=2), axis=1)
    masks = np.zeros((total_samples, MAX_CHOICES), dtype='float32')
    for i, count in enumerate(non_zero_counts): masks[i, :count] = 1.0

    final_dataset = (contexts_final, choices_final, labels_final, masks)

    print(f"Saving final dataset to Google Drive: '{FINAL_DATASET_PATH_GDRIVE}'...")
    with open(FINAL_DATASET_PATH_GDRIVE, 'wb') as f:
        pickle.dump(final_dataset, f, protocol=pickle.HIGHEST_PROTOCOL)

    print("\n--- Final Dataset Created Successfully! --- AIの学習に進んでください。 ---")

    # 一時ファイルをクリーンアップ
    MERGED_LOG_PATH = os.path.join(GDRIVE_PROCESSED_DIR, 'merged_chunks.log')
    !rm -f "{CONTEXTS_BIN_PATH}"
    !rm -f "{CHOICES_BIN_PATH}"
    !rm -f "{LABELS_BIN_PATH}"
    !rm -f "{MERGED_LOG_PATH}"
    print("Cleaned up temporary files.")

## STEP 4: AIモデルの学習

**STEP 3が完了したら、このセルを実行してください。**

In [ ]:
import os

#【安全対策】実行前に必ずプロジェクトルートに移動します
%cd {GDRIVE_PROJECT_ROOT}

# train_transformer.pyをDrive上で動作するように、パスを書き換えます
with open('src/transformer/train_transformer.py', 'r') as f:
    code = f.read()
code = code.replace("PROCESSED_DATA_PATH = '/content/processed_data/training_dataset_transformer.pkl'", f"PROCESSED_DATA_PATH = '{os.path.join(GDRIVE_PROJECT_ROOT, 'processed_data', 'training_dataset_transformer.pkl')}'")
code = code.replace("MODEL_PATH = '/content/models/senas_jan_ai_transformer_v1.keras'", f"MODEL_PATH = '{os.path.join(GDRIVE_PROJECT_ROOT, 'models', 'senas_jan_ai_transformer_v1.keras')}'")
code = code.replace("os.makedirs('/content/models', exist_ok=True)", f"os.makedirs('{os.path.join(GDRIVE_PROJECT_ROOT, 'models')}', exist_ok=True)")
with open('src/transformer/train_transformer_drive.py', 'w') as f:
    f.write(code)

print("--- Starting Training ---")
!python -m src.transformer.train_transformer_drive

## STEP 5: 学習済みAIによる予測

**学習が完了したら、このセルを実行してください。**

In [ ]:
import os

#【安全対策】実行前に必ずプロジェクトルートに移動します
%cd {GDRIVE_PROJECT_ROOT}

# predict_transformer.pyをDrive上で動作するように、パスを書き換えます
with open('src/transformer/predict_transformer.py', 'r') as f:
    code = f.read()
code = code.replace("MODEL_PATH = '/content/models/senas_jan_ai_transformer_v1.keras'", f"MODEL_PATH = '{os.path.join(GDRIVE_PROJECT_ROOT, 'models', 'senas_jan_ai_transformer_v1.keras')}'")
with open('src/transformer/predict_transformer_drive.py', 'w') as f:
    f.write(code)

!python -m src.transformer.predict_transformer_drive